# Volume 1, Chapter 1: What Is Generative AI?

**Your first hands-on experience with LLMs for networking tasks**

From: AI for Networking and Security Engineers - Volume 1, Chapter 1

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eduardd76/AI_for_networking_and_security_engineers/blob/master/Volume-1-Foundations/Colab-Notebooks/Vol1_Ch1_Generative_AI.ipynb)

---

**What you'll experience:**
- Make your **first API call** to Claude and see token usage/cost
- AI analyzing a router config -- finding issues like a senior engineer
- **Traditional regex vs AI parsing** -- side-by-side comparison (IOS + JunOS)
- System prompts, temperature, and how they change behavior
- Config sanitization -- security first
- Multi-turn troubleshooting conversation with memory
- Config generation from natural language requirements

**Time:** ~20 minutes | **Cost:** ~$0.10 in API calls

## Setup

Install the Anthropic Python SDK and set your API key.
Get a free API key at [console.anthropic.com](https://console.anthropic.com/).

In [ ]:
# Install the Anthropic Python SDK
!pip install -q anthropic

import os
import json
import re
from getpass import getpass
from datetime import datetime

# Set up API key -- try Colab Secrets first, then manual entry
try:
    from google.colab import userdata
    os.environ['ANTHROPIC_API_KEY'] = userdata.get('ANTHROPIC_API_KEY')
    print('API key loaded from Colab Secrets')
except Exception:
    if 'ANTHROPIC_API_KEY' not in os.environ:
        os.environ['ANTHROPIC_API_KEY'] = getpass('Enter your Anthropic API key: ')
    print('API key set manually')

from anthropic import Anthropic

# Initialize the client -- it reads ANTHROPIC_API_KEY automatically
client = Anthropic()

# Which model to use. Sonnet = best balance of quality and cost.
MODEL = "claude-sonnet-4-20250514"

print(f"Using model: {MODEL}")
print("Setup complete. Ready to make your first API call!")

## Demo 1: Your First API Call -- "Hello, Network World"

The simplest possible interaction: ask Claude a networking question.

This is the same request/response pattern as calling any REST API
(Meraki Dashboard, DNA Center, etc.). HTTPS POST in, JSON out.

In [ ]:
# -------------------------------------------------------------------
# Your first API call -- as simple as it gets
# -------------------------------------------------------------------

response = client.messages.create(
    model=MODEL,
    max_tokens=512,
    messages=[{
        "role": "user",
        "content": "Explain what OSPF is in 3 sentences, as if explaining to a junior network engineer."
    }]
)

# Print the response
print("Claude's response:")
print("=" * 60)
print(response.content[0].text)
print("=" * 60)

# Token usage -- this is how you track costs
print(f"\nToken usage:")
print(f"  Input tokens:  {response.usage.input_tokens}")
print(f"  Output tokens: {response.usage.output_tokens}")
print(f"  Total tokens:  {response.usage.input_tokens + response.usage.output_tokens}")
print(f"\nEstimated cost (Sonnet): ${(response.usage.input_tokens * 3 + response.usage.output_tokens * 15) / 1_000_000:.6f}")

## Demo 2: Analyzing a Router Configuration

This is where it gets useful. We give Claude a router config and ask it to
review it -- the same thing a senior engineer would do during a config review,
but in seconds instead of minutes.

**What's happening**: We send the config as text in the API request.
The prompt tells Claude what to look for. Claude reads the config, applies its
understanding of networking best practices, and returns a structured analysis.

In [ ]:
# -------------------------------------------------------------------
# Sample Cisco IOS router config (with intentional issues)
# In production, you'd read this from a file or pull it via Netmiko
# -------------------------------------------------------------------

sample_config = """
hostname router-core-01
!
interface Loopback0
 ip address 192.168.1.1 255.255.255.255
!
interface GigabitEthernet0/0
 description Uplink to ISP-A (Lumen)
 ip address 203.0.113.1 255.255.255.252
 no shutdown
!
interface GigabitEthernet0/1
 description Trunk to DC-SPINE-01
 ip address 10.0.1.1 255.255.255.0
 no shutdown
!
interface GigabitEthernet0/2
 description Link to DR-SITE -- INACTIVE
 ip address 10.0.2.1 255.255.255.0
 shutdown
!
interface GigabitEthernet0/3
 no description
 ip address 172.16.50.1 255.255.255.0
 no shutdown
!
router ospf 1
 router-id 192.168.1.1
 network 10.0.0.0 0.0.255.255 area 0
 network 192.168.1.1 0.0.0.0 area 0
 passive-interface default
 no passive-interface GigabitEthernet0/1
!
router bgp 65001
 bgp router-id 192.168.1.1
 bgp log-neighbor-changes
 neighbor 203.0.113.2 remote-as 65002
 neighbor 203.0.113.2 description ISP-A_Lumen
 neighbor 203.0.113.2 route-map ISP-A-IN in
 neighbor 203.0.113.2 route-map ISP-A-OUT out
!
ip prefix-list ADVERTISE-TO-ISP seq 10 permit 198.51.100.0/24
!
route-map ISP-A-OUT permit 10
 match ip address prefix-list ADVERTISE-TO-ISP
!
route-map ISP-A-IN permit 10
 set local-preference 200
!
ip access-list extended MANAGEMENT_ACCESS
 permit tcp 10.0.0.0 0.0.255.255 any eq 22
 permit tcp 172.16.0.0 0.0.0.255 any eq 22
 deny ip any any log
!
line vty 0 4
 access-class MANAGEMENT_ACCESS in
 transport input ssh
"""

print(f"Config loaded: {len(sample_config)} characters")
print(f"Preview:\n{sample_config[:200]}...")

In [ ]:
# -------------------------------------------------------------------
# Ask Claude to review the config like a senior engineer
# -------------------------------------------------------------------

response = client.messages.create(
    model=MODEL,
    max_tokens=2048,
    system="You are a senior network engineer performing a configuration review.",
    messages=[{
        "role": "user",
        "content": f"""Review this router configuration and provide:

1. **Security concerns** -- anything that could be a vulnerability
2. **Best practices followed** -- what's done well
3. **Operational risks** -- things that could cause outages or issues
4. **Recommendations** -- specific improvements with example commands

Configuration:
{sample_config}"""
    }]
)

print("CONFIG REVIEW RESULTS")
print("=" * 60)
print(response.content[0].text)
print("=" * 60)
print(f"\nTokens used: {response.usage.input_tokens} in / {response.usage.output_tokens} out")
print(f"Cost: ${(response.usage.input_tokens * 3 + response.usage.output_tokens * 15) / 1_000_000:.4f}")

## Demo 3: Understanding Tokens (The "Packets" of AI)

Tokens are to LLMs what packets are to networks. Understanding them helps you:
- **Budget costs** (you pay per token)
- **Fit within limits** (context window = max tokens per request)
- **Write efficient prompts** (fewer tokens = faster, cheaper)

Let's see how networking text gets tokenized and what it costs.

In [ ]:
# -------------------------------------------------------------------
# Token estimation for networking text
# -------------------------------------------------------------------

test_texts = {
    "Simple sentence": "The router is forwarding packets correctly.",
    "CLI command": "show ip bgp summary | include Established",
    "IP address": "192.168.255.1",
    "Interface name": "GigabitEthernet0/0/0.100",
    "Config block": """router ospf 1
 router-id 10.0.0.1
 network 10.0.0.0 0.0.255.255 area 0
 passive-interface default""",
    "ACL entry": "permit tcp 10.0.0.0 0.0.255.255 host 192.168.1.100 eq 443 log",
    "Full config (sample)": sample_config
}

print("TOKEN ESTIMATION FOR NETWORKING TEXT")
print("=" * 70)
print(f"{'Text Type':<25} {'Characters':<12} {'Est. Tokens':<12} {'Est. Cost*'}")
print("-" * 70)

for name, text in test_texts.items():
    chars = len(text)
    # Rough estimate: configs average ~3.5 chars/token
    est_tokens = max(1, chars // 4)
    cost = est_tokens * 3 / 1_000_000
    print(f"{name:<25} {chars:<12} ~{est_tokens:<11} ${cost:.6f}")

print("\n* Input cost at Sonnet rate ($3/1M tokens)")
print("\nKey insight: A full router config (~1000 lines) is roughly")
print("3,000-5,000 tokens, costing about $0.01-0.02 to analyze with Sonnet.")
print("That's analyzing 100 devices for about $1-2.")

## Demo 4: Traditional Automation vs. Generative AI

Same problem, two approaches: **"Extract interface information from a config."**

- **Traditional**: Write regex patterns to parse specific fields
- **Generative AI**: Ask Claude to read the config and extract what we need

Both work. But watch what happens when the format changes (IOS vs JunOS).

In [ ]:
# -------------------------------------------------------------------
# APPROACH 1: Traditional parsing with regex
# -------------------------------------------------------------------

def parse_interfaces_regex(config):
    """Traditional approach: parse interfaces with regex.
    
    Works for IOS, but:
    - Needs different regex for each vendor (NX-OS, JunOS, EOS)
    - Edge cases break it (missing fields, unexpected formatting)
    - Adding new fields = writing new patterns
    """
    interfaces = []
    pattern = r'interface (\S+)\n((?:[ !].*\n)*)'
    matches = re.finditer(pattern, config)
    
    for match in matches:
        name = match.group(1)
        block = match.group(2)
        ip_match = re.search(r'ip address (\S+ \S+)', block)
        desc_match = re.search(r'description (.+)', block)
        shutdown = 'shutdown' in block and 'no shutdown' not in block
        
        interfaces.append({
            'name': name,
            'ip': ip_match.group(1) if ip_match else 'N/A',
            'description': desc_match.group(1).strip() if desc_match else 'N/A',
            'admin_status': 'down' if shutdown else 'up'
        })
    
    return interfaces

print("APPROACH 1: Traditional Regex Parsing")
print("=" * 60)
parsed = parse_interfaces_regex(sample_config)
for iface in parsed:
    print(f"  {iface['name']:<30} {iface['ip']:<25} {iface['admin_status']:<6} {iface['description']}")

print(f"\nFound {len(parsed)} interfaces")
print("\nLimitation: This regex is IOS-specific. Won't work on")
print("NX-OS, JunOS, or Arista EOS without a rewrite.")

In [ ]:
# -------------------------------------------------------------------
# APPROACH 2: Generative AI -- ask Claude to parse the config
# -------------------------------------------------------------------

def parse_interfaces_ai(config):
    """AI approach: ask Claude to extract interface data.
    
    Advantages:
    - Works across vendors (IOS, NX-OS, JunOS, EOS)
    - Handles edge cases and unusual formatting
    - Easy to add new fields (just update the prompt)
    
    Trade-offs:
    - Slower (API call vs local regex)
    - Costs money (fractions of a cent per call)
    - Not 100% deterministic
    """
    response = client.messages.create(
        model=MODEL,
        max_tokens=1500,
        temperature=0,
        messages=[{
            "role": "user",
            "content": f"""Extract all interfaces from this config as a JSON array.

For each interface include:
- name: interface name
- ip: IP address with mask (or "N/A")
- description: interface description (or "N/A")
- admin_status: "up" or "down" (based on shutdown/no shutdown)
- ospf: true/false (is this interface in OSPF?)
- vrf: VRF name if assigned, otherwise "global"

Return ONLY valid JSON, no other text.

Config:
{config}"""
        }]
    )
    
    text = response.content[0].text.strip()
    text = re.sub(r'^```(?:json)?\s*', '', text)
    text = re.sub(r'\s*```$', '', text)
    return json.loads(text), response.usage


print("APPROACH 2: Generative AI Parsing")
print("=" * 60)
ai_parsed, usage = parse_interfaces_ai(sample_config)

for iface in ai_parsed:
    ospf = "OSPF" if iface.get('ospf') else ""
    vrf = iface.get('vrf', 'global')
    print(f"  {iface['name']:<30} {iface['ip']:<25} {iface['admin_status']:<6} {ospf:<5} {iface['description']}")

print(f"\nFound {len(ai_parsed)} interfaces")
print(f"Tokens: {usage.input_tokens} in / {usage.output_tokens} out")
print(f"Cost: ${(usage.input_tokens * 3 + usage.output_tokens * 15) / 1_000_000:.4f}")
print("\nNotice: AI also extracted OSPF participation and VRF -- fields")
print("that would need additional regex in the traditional approach.")

In [ ]:
# -------------------------------------------------------------------
# The AI advantage: same code works on Juniper JunOS -- zero changes
# -------------------------------------------------------------------

junos_config = """
interfaces {
    ge-0/0/0 {
        description "Uplink to ISP-B (Cogent)";
        unit 0 {
            family inet {
                address 198.51.100.1/30;
            }
        }
    }
    ge-0/0/1 {
        description "Trunk to DC-LEAF-01";
        unit 0 {
            family inet {
                address 10.1.1.1/24;
            }
        }
    }
    lo0 {
        unit 0 {
            family inet {
                address 192.168.2.1/32;
            }
        }
    }
}
protocols {
    ospf {
        area 0.0.0.0 {
            interface ge-0/0/1.0;
            interface lo0.0 {
                passive;
            }
        }
    }
}
"""

print("SAME AI FUNCTION ON JUNOS CONFIG")
print("=" * 60)
print("(No code changes needed -- the AI understands both vendors)\n")

junos_parsed, usage = parse_interfaces_ai(junos_config)

for iface in junos_parsed:
    ospf = "OSPF" if iface.get('ospf') else ""
    print(f"  {iface['name']:<30} {iface.get('ip','N/A'):<25} {iface.get('admin_status','up'):<6} {ospf:<5} {iface.get('description','N/A')}")

print(f"\nTokens: {usage.input_tokens} in / {usage.output_tokens} out")
print(f"Cost: ${(usage.input_tokens * 3 + usage.output_tokens * 15) / 1_000_000:.4f}")
print("\nKey insight: The regex parser from Approach 1 would need a complete")
print("rewrite for JunOS. The AI parser works without any changes.")

## Demo 5: The System Prompt -- Your "Route-Map" for AI

The **system prompt** shapes ALL of the model's responses.
It's applied to every message, like a route-map applied to a BGP peer.

Watch how the same question gets different answers with different system prompts.

In [ ]:
# -------------------------------------------------------------------
# Same question, different system prompts = different responses
# -------------------------------------------------------------------

question = "Should I use OSPF or BGP for my data center fabric?"

system_prompts = {
    "No system prompt": None,
    "Junior-friendly": "You are a patient network mentor. Explain concepts simply, "
                       "using analogies. Assume the reader is new to data center networking.",
    "Senior engineer": "You are a senior network architect at a hyperscaler. Be direct, "
                       "technical, and opinionated. Cite specific scale and convergence numbers.",
    "Vendor-neutral consultant": "You are a vendor-neutral network consultant. Always present "
                                  "both sides fairly. Include specific use cases where each option wins."
}

for label, system in system_prompts.items():
    print(f"\n{'='*60}")
    print(f"SYSTEM PROMPT: {label}")
    print(f"{'='*60}")
    
    kwargs = {
        "model": MODEL,
        "max_tokens": 300,
        "messages": [{"role": "user", "content": question}]
    }
    if system:
        kwargs["system"] = system
    
    response = client.messages.create(**kwargs)
    print(response.content[0].text)
    print(f"\n[Tokens: {response.usage.input_tokens} in / {response.usage.output_tokens} out]")

## Demo 6: Temperature -- Controlling Randomness

**Temperature** controls how "creative" vs. "deterministic" the model is.

- `temperature=0`: Most deterministic. Same input -> (nearly) same output.
  Use for config generation, data extraction, factual answers.
- `temperature=1`: Most creative/varied.
  Use for brainstorming, generating multiple design options.

**Networking analogy**: Temperature is like jitter. At 0, every packet
takes the exact same path. At 1, there's ECMP-style randomness.

In [ ]:
# -------------------------------------------------------------------
# Temperature comparison: deterministic vs. creative
# -------------------------------------------------------------------

prompt = "Generate a hostname for a core router in the Chicago data center."

print("TEMPERATURE = 0 (deterministic)")
print("Running 3 times -- should give similar results:")
print("-" * 40)
for i in range(3):
    resp = client.messages.create(
        model=MODEL, max_tokens=50, temperature=0,
        messages=[{"role": "user", "content": prompt}]
    )
    print(f"  Run {i+1}: {resp.content[0].text.strip()}")

print(f"\nTEMPERATURE = 1.0 (creative)")
print("Running 3 times -- expect different results:")
print("-" * 40)
for i in range(3):
    resp = client.messages.create(
        model=MODEL, max_tokens=50, temperature=1.0,
        messages=[{"role": "user", "content": prompt}]
    )
    print(f"  Run {i+1}: {resp.content[0].text.strip()}")

print("\nRule of thumb:")
print("  Config generation / data extraction: temperature=0")
print("  Creative tasks / brainstorming:      temperature=0.7-1.0")
print("  Analysis / review:                   temperature=0")

## Demo 7: Config Sanitization -- Security First

Before sending any config to an external API, you **must** remove sensitive data.
This function strips passwords, SNMP communities, TACACS keys, and other secrets.

**Rule**: Never send credentials to any external service.

In [ ]:
# -------------------------------------------------------------------
# Config sanitizer: remove secrets before sending to any API
# -------------------------------------------------------------------

def sanitize_config(config: str) -> str:
    """Remove sensitive data from network configs before sending to AI.
    
    Strips: passwords, SNMP communities, TACACS/RADIUS keys,
    crypto keys, and other secrets.
    """
    sanitized = config
    patterns = [
        (r'(password|secret)\s+\d*\s*\S+', r'\1 *** REDACTED ***'),
        (r'(snmp-server community)\s+\S+', r'\1 REDACTED'),
        (r'(tacacs-server key)\s+\S+', r'\1 REDACTED'),
        (r'(radius-server key)\s+\S+', r'\1 REDACTED'),
        (r'(key-string)\s+\S+', r'\1 REDACTED'),
        (r'(crypto isakmp key)\s+\S+', r'\1 REDACTED'),
        (r'(enable secret)\s+\d*\s*\S+', r'\1 *** REDACTED ***'),
        (r'(username \S+ (?:privilege \d+ )?(?:secret|password))\s+\d*\s*\S+',
         r'\1 *** REDACTED ***'),
        (r'(wpa-psk ascii)\s+\S+', r'\1 REDACTED'),
        (r'(api-key|token|auth-key)\s+\S+', r'\1 REDACTED'),
    ]
    for pattern, replacement in patterns:
        sanitized = re.sub(pattern, replacement, sanitized, flags=re.IGNORECASE)
    return sanitized


# Test with a config that has secrets
config_with_secrets = """
hostname router-edge-01
!
enable secret 5 $1$mERr$hx5rVt7rPNoS4wqbXKX7m0
!
username admin privilege 15 secret 0 SuperSecret123!
username readonly password 0 ReadOnly456
!
snmp-server community N3tw0rkR0cks RO
snmp-server community Wr1t3M3In RW
!
tacacs-server key MyTacacsKey2024
!
interface GigabitEthernet0/0
 description WAN Uplink to ISP
 ip address 203.0.113.5 255.255.255.252
!
crypto isakmp key VpnPreShared123 address 198.51.100.1
"""

sanitized = sanitize_config(config_with_secrets)

print("BEFORE SANITIZATION (contains secrets):")
print("-" * 50)
print(config_with_secrets)
print("\nAFTER SANITIZATION (safe to send to API):")
print("-" * 50)
print(sanitized)

## Demo 8: Multi-Turn Conversation -- Troubleshooting Session

LLMs maintain conversation context across multiple messages.
This lets you have a back-and-forth troubleshooting session,
just like working with a colleague on a bridge call.

Each message builds on the previous ones -- the model remembers
everything discussed so far.

In [ ]:
# -------------------------------------------------------------------
# Multi-turn troubleshooting conversation
# -------------------------------------------------------------------

class NetworkTroubleshooter:
    """Multi-turn conversation wrapper for troubleshooting.
    Maintains history so AI remembers context -- like a persistent SSH session.
    """
    def __init__(self):
        self.history = []
        self.system = (
            "You are a senior network engineer helping troubleshoot issues. "
            "Ask clarifying questions when needed. Be concise but thorough. "
            "When you identify the likely cause, explain your reasoning."
        )
    
    def ask(self, message):
        self.history.append({"role": "user", "content": message})
        response = client.messages.create(
            model=MODEL, max_tokens=1024,
            system=self.system, messages=self.history
        )
        reply = response.content[0].text
        self.history.append({"role": "assistant", "content": reply})
        return reply


# Simulate a troubleshooting session
ts = NetworkTroubleshooter()

conversation = [
    "We're seeing intermittent packet loss between our core router (10.0.0.1) "
    "and the distribution switch (10.0.0.2). It's been happening for 2 days.",
    
    "The link is a 10G fiber connection. show interface shows: \n"
    "  input errors: 1,247\n"
    "  CRC errors: 1,245\n"
    "  output errors: 0\n"
    "  input rate: 3.2 Gbps\n"
    "  output rate: 2.8 Gbps",
    
    "The SFP was replaced 3 months ago. The fiber run is about 150 meters. "
    "show interfaces transceiver shows: \n"
    "  TX power: -2.1 dBm\n"
    "  RX power: -14.8 dBm\n"
    "  Temperature: 72C"
]

for i, msg in enumerate(conversation, 1):
    print(f"\n{'='*60}")
    print(f"ENGINEER (message {i}):")
    print(f"{'='*60}")
    print(msg)
    
    response = ts.ask(msg)
    print(f"\nAI ASSISTANT:")
    print("-" * 60)
    print(response)

print(f"\n\nConversation: {len(ts.history)} messages exchanged.")
print("The AI remembers all context from previous messages.")

## Demo 9: Generating Configurations from Requirements

Describe what you need in English, get back a configuration.
Great for vendor syntax you don't use daily, boilerplate configs,
or converting requirements docs into actual configs.

**Critical rule**: NEVER apply AI-generated configs to production without review.
Treat them like code from a junior engineer -- probably correct, but verify.

In [ ]:
# -------------------------------------------------------------------
# Generate a config from natural language requirements
# -------------------------------------------------------------------

requirements = """
I need a Cisco IOS configuration for an access switch with these requirements:

1. Hostname: sw-access-floor3-01
2. Three VLANs:
   - VLAN 10: Employees (10.10.10.0/24)
   - VLAN 20: VoIP phones (10.10.20.0/24)
   - VLAN 99: Management (10.10.99.0/24)
3. Ports Gi1/0/1-24: access ports for employees with voice VLAN for phones
4. Ports Gi1/0/47-48: trunk uplinks to distribution switch
5. Port security: max 3 MAC addresses per port, sticky learning, shutdown on violation
6. DHCP snooping enabled for VLANs 10 and 20 (trust uplinks)
7. Spanning tree: rapid-PVST, root guard on access ports, BPDU guard on access ports
8. Management VLAN 99 SVI with IP 10.10.99.10/24
9. SSH-only management, no telnet
10. NTP server: 10.10.99.1
"""

response = client.messages.create(
    model=MODEL,
    max_tokens=3000,
    temperature=0,
    system="You are a network engineer generating production-quality configurations. "
           "Include comments explaining each section. Only output the configuration.",
    messages=[{"role": "user", "content": requirements}]
)

print("GENERATED CONFIGURATION")
print("=" * 60)
print(response.content[0].text)
print("=" * 60)
print(f"\nTokens: {response.usage.input_tokens} in / {response.usage.output_tokens} out")
print(f"Cost: ${(response.usage.input_tokens * 3 + response.usage.output_tokens * 15) / 1_000_000:.4f}")
print("\nALWAYS REVIEW before applying to production!")

## Try It Yourself!

Use the cell below to experiment. Some ideas:

- Paste a config snippet and ask for a review
- Ask Claude to explain a networking concept
- Request a config for a specific scenario
- Ask it to convert IOS config to JunOS
- Give it a log message and ask for root cause analysis

Remember to sanitize any real configs before pasting!

In [ ]:
# -------------------------------------------------------------------
# YOUR TURN: Modify the prompt below and run this cell
# -------------------------------------------------------------------

your_prompt = """
Explain the difference between VXLAN and traditional VLANs.
Use an analogy that a network engineer would appreciate.
Keep it under 200 words.
"""

# Optional: customize the system prompt
your_system = "You are a helpful network engineering assistant."

response = client.messages.create(
    model=MODEL,
    max_tokens=1024,
    system=your_system,
    messages=[{"role": "user", "content": your_prompt}]
)

print("RESPONSE")
print("=" * 60)
print(response.content[0].text)
print("=" * 60)
print(f"Tokens: {response.usage.input_tokens} in / {response.usage.output_tokens} out")
print(f"Cost: ${(response.usage.input_tokens * 3 + response.usage.output_tokens * 15) / 1_000_000:.4f}")

## Summary

This notebook demonstrated the core concepts from Chapter 1:

1. **First API Call** -- Sending a question to Claude (same as any REST API)
2. **Config Analysis** -- AI reviewing a router config like a senior engineer
3. **Token Counting** -- Understanding the "packets" of AI: measurement and pricing
4. **Traditional vs. AI Parsing** -- Regex works for known formats; AI works across vendors
5. **Cross-Vendor Parsing** -- Same AI function handles IOS and JunOS with zero changes
6. **System Prompts** -- Shaping AI behavior (like a route-map for all interactions)
7. **Temperature** -- Controlling determinism vs. creativity
8. **Config Sanitization** -- Security first: strip secrets before sending to any API
9. **Multi-Turn Conversations** -- Troubleshooting sessions with full memory
10. **Config Generation** -- Requirements in English, configs out

### Key Takeaways

| Traditional Automation | AI-Powered Automation |
|----------------------|----------------------|
| Regex patterns | Semantic understanding |
| Vendor-specific parsers | Works across vendors |
| Can't explain "why" | Provides reasoning |
| Manual rule updates | Learns from context |
| Exact matches only | Understands intent |

- **Cost is minimal**: Analyzing a router config costs fractions of a cent
- **AI complements automation**: Use regex for known tasks, AI for reasoning and cross-vendor work
- **Always sanitize configs** before sending to external APIs
- **Never trust AI output blindly** -- verify before deploying

### Next Steps
- **Chapter 2**: How LLMs work (transformer architecture, tokenization)
- **Chapter 3**: Choose the right model for your use case
- **Chapter 4**: API authentication, error handling, production patterns

-> [Continue to Chapter 2 Notebook](./Vol1_Ch2_Tokenizer.ipynb)